In [2]:
import pandas as pd
import re,nltk


In [3]:
from sklearn.model_selection import train_test_split as tts
from sklearn import pipeline
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize.treebank import TreebankWordDetokenizer
import numpy as np

In [4]:
data=pd.read_csv('/Users/aadiharan99/Reviews.csv')
data=data.sample(1000)

In [5]:
from nltk.corpus import stopwords
sw=set(stopwords.words('english'))


In [6]:
def word_preprocessor(text):
  text=re.sub('[^a-zA-Z]', " ", str(text))
  text=text.lower()
  text_tokens=word_tokenize(text)
  text_tokens_no_sw=[w for w in text_tokens if not w in sw]
  text=TreebankWordDetokenizer().detokenize(text_tokens_no_sw)
  return text
  
  

In [7]:
data['cleaned_text']=data['Text'].apply(word_preprocessor)

In [8]:
data.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,cleaned_text
519343,519344,B002J9HQIY,A1EPU3WYJVO2BZ,Browneyedgirl,3,3,4,1316217600,Overpriced,I got the same product at my local grocery sto...,got product local grocery store much cheaper g...
410580,410581,B001KUOH26,AKCZIJ6ZHE41D,"Jarucia Jaycox Nirula ""~A Pink American""",8,8,5,1227225600,"Bye-Bye Zeros, Hello My New Hero--Zevia",I grew up drinking Diet sodas. No joke. I'd ...,grew drinking diet sodas joke never tasted reg...
556227,556228,B004EDZ87Y,A4VMQ6ZTSXSSL,"kiwanissandy ""Go Bucks!""",27,28,5,1292457600,"Delicious with 5 spice aromas, not full of sug...",This India Spice Chai while brewing fills the ...,india spice chai brewing fills whole room wond...
506254,506255,B00413AELW,A2XT1DFEV6UYHX,L. SVAC,0,0,5,1292716800,Holiday Tree Dried Fruit Gift Package,The recipients of this gift were delighted wit...,recipients gift delighted package arrived sche...
318568,318569,B000NGP8UI,A1LIM4V7Q6LJJJ,N. Cao,2,5,1,1272153600,New formula is absurd!,When I found out Rockstar Juiced recently chan...,found rockstar juiced recently changed formula...


In [9]:
data.drop(['Text'],1,inplace=True)

In [114]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf_idf=TfidfVectorizer()

In [115]:
score=tf_idf.fit_transform(data['cleaned_text'])

In [116]:
dense=score.todense()
dense1=dense.tolist()

In [118]:
col_names=tf_idf.get_feature_names()

In [119]:
df = pd.DataFrame(dense1,columns = col_names)

In [120]:
df.head()

,abated,ability,able,abouth,abroad,absence,absolute,absolutely,absolutly,absorb,...,zing,zinger,zingiber,zingy,zip,ziplock,zmy,zoe,zone,zuke
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [121]:
import numpy as np
from sklearn.decomposition import NMF

In [147]:
nmf=NMF(10,random_state=42,alpha=0.1,l1_ratio=0.5,init='nndsvd')

In [148]:
arr=nmf.fit_transform(df)

In [149]:
comps=nmf.components_

In [150]:
comps

array([[0.        , 0.        , 0.00300981, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.00807306, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.02621055]])

In [151]:
from sklearn.decomposition import LatentDirichletAllocation

In [152]:
lda=LatentDirichletAllocation(n_components=10)

In [153]:
model1=lda.fit_transform(df)

In [154]:
comps1=lda.components_

In [155]:
def print_topics(model, count_vectorizer, n_top_words):
    words = count_vectorizer.get_feature_names()
    for topic_idx, topic in enumerate(model.components_):
        print("\nTopic #%d:" % topic_idx)
        print(" ".join([words[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))

In [156]:
print_topics(lda,tf_idf,100)


Topic #0:
br product dog like food tea loves oil great add com href gp www http one sugar use well much training husband better amazon cereal ingredients new treats butter flavor love good eat taste pack coffee dogs different would price noodles best treat chai wish excellent free cup back pretty take making get quality seems smell coconut healthy without bag since however local seemed stars puppy son artificial breakfast snacks leaf dry cough even peanut energy fruit organic bought cat orange crackers bars kinds jerky calories colors though beef size contain regular high ever problems far bottle say version favorite

Topic #1:
br like tea good bread soup gluten used favorite bad chocolate time free made get filling win sugar eat dog really right breakfast coconut food stuff protein baking crunch tastes milk real something product benefits would black oatmeal sauce quite great yummy taste cup hoisin picky bought cream home bar type espresso surely probably one cans mix kids go power l

In [157]:
print_topics(nmf,tf_idf,100)



Topic #0:
like good taste one really would flavor product chocolate get also tried much time love little eat buy try better even way first sugar best use lot tastes got bought well go thing day bag make flavors sweet milk bars hot box two used think calories recommend something eating healthy favorite right made give whole though order brand organic tasted free another bad coconut texture thought definitely drink know bit real stuff say quite ordered snack fresh without delicious every highly want buying easy fat bar could money low tasty loved looking never nice since pasta put add fruit makes

Topic #1:
tea green bags chai black teas breakfast love cup drink leaf brew strong english stash iced drinking wonderful full flavor add sugar cold twinings go leaves needed quality drinker tazo flowers every hint brewed make afternoon sweet honey aroma favorite numi balance absolutely loose dilmah freinds little spice hot weak perfect deliciozoooo flavored steep list gifts wrong mint herbal a